# 中文分词技术

## 规则分词

### 正向最大匹配法

### 逆向最大匹配法

### 双向最大匹配法

## 统计分词

<b>步骤</b>：

1. 建立统计语言模型。

2. 对句子进行单词划分，然后对划分结果进行概率计算，获得概率最大的分词方式。这里就用到了统计学习方法，如隐马尔科夫模型（HMM）、条件随机场（CRF）等。

### 语言模型

<b>n元模型（n-gram model）</b>：
- n=1时，一元模型（unigram model）
- n=2时，二元模型（bigram model）
- n=3时，三元模型（trigram model）

### HMM模型

基本思路：    
每个字在构造一个特定的词语时都占据着一个确定的构词位置（词位），规定每个字最多只有四个构词位置：B（词首）、M（词中）、E（词尾）、S（单独成词）。

In [ ]:
#定义HMM类
class HMM(object):
    #构造函数
    def __init__(self):
        import os
        #存取算法中间结果
        self.model_file = './data/chapter3/hmm_model.pkl'
        #状态值集合
        self.state_list = ['B','M','E','S']
        #参数加载，判断是否需要重新加载model_file
        self.load_para = False
    
    #用于加载已计算的中间结果，当需要重新训练时，需要初始化清空结果
    def try_load_model(self,trained):
        if trained:
            import pickle
            with open(self.model_file,'rb') as f:
                self.A_dic = pickle.load(f)
                self.B_dic = pickle.load(f)
                self.Pi_dic = pickle.load(f)
                self.load_para = True
        else:
            #状态转移概率（状态->状态的条件概率）
            self.A_dic = {}
            #发射概率（状态->词语的条件概率）
            self.B_dic = {}
            #状态的初始概率
            self.Pi_dic = {}
            self.load_para = False
    
    #计算转移概率、发射概率以及初始概率
    def train(self,path):
        #重置三个概率矩阵
        self.try_load_model(False)
        #统计状态出现次数，求p(o)
        Count_dic = {}
        #初始化参数
        def init_parameters():
            for state in self.state_list:
                self.A_dic[state] = {s:0.0 for s in self.state_list}
                self.B_dic[state] = {}
                self.Pi_dic[state] = 0.0
                Count_dic[state] = 0
        #为每个字标记状态（构词位置）
        def makeLabel(text):
            out_text = []
            if len(text) == 1:
                out_text.append('S')
            else:
                out_text += ['B'] + ['M'] * (len(text)-2) + ['E']
            return out_text
        init_parameters()
        line_num = -1
        #
        words = set()
        with open(path,encoding='utf8') as f:
            for line in f:
                line_num += 1
                line = line.strip()
                if not line:
                    continue
                word_list = [i for i in line if i != ' ']
                words |= set(word_list) #更新字的集合
                linelist = line.split()
                line_state = []
                for w in linelist:
                    line_state.extend(makeLabel(w))
                assert len(word_list) == len(line_state), len(word_list)/len(line_state)
                for k,v in enumerate(line_state):
                    Count_dic[v] += 1
                    if k == 0:
                        self.Pi_dic[v] += 1    #每个句子第一个字的状态，用于计算初始状态概率
                    else:
                        self.A_dic[line_state[k-1]][v] += 1 #计算转移概率
                        self.B_dic[line_state[k]][word_list[k]] = self.B_dic[line_state[k]].get(word_list[k],0) + 1.0 #计算发射概率

            self.Pi_dic = {k : v * 1.0 / line_num for k,v in self.Pi_dic.items() }
            self.A_dic = {k : {k1 : v1  / Count_dic[k] for k1,v1 in v.items()} for k,v in self.A_dic.items()}
            self.B_dic = {k : {k1 : (v1 + 1)  / Count_dic[k] for k1,v1 in v.items()} for k,v in self.B_dic.items()}
            
            import pickle
            with open(self.model_file, 'wb') as f:
                pickle.dump(self.A_dic, f)
                pickle.dump(self.B_dic, f)
                pickle.dump(self.Pi_dic, f)
            return self
    
    #Viterbi算法实现
    def viterbi(self,text,states,start_p,trans_p,emit_p):
        V = [{}]        #
        path = {}       #
        #句首第一个字的状态概率分布
        for y in states:
            V[0][y] = start_p[y]*emit_p[y].get(text[0],0)
            path[y] = [y]
        #第二个字开始的状态概率分布    
        for t in range(1,len(text)):
            V.append({})
            newpath = {}
            #检验训练的发射概率矩阵中是否有该字
            neverSeen = text[t] not in emit_p['S'].keys() and text[t] not in emit_p['B'].keys() and text[t] not in emit_p['M'].keys() and text[t] not in emit_p['E'].keys()
            for y in states:
                emitP = emit_p[y].get(text[t],0) if not neverSeen else 1.0
                (prob,state) = max([(V[t-1][y0] * trans_p[y0].get(y,0) * emitP , y0) for y0 in states if V[t-1][y0] > 0])
                V[t][y] = prob
                newpath[y] = path[state] + [y]
            path = newpath
        if emit_p['M'].get(text[-1],0) > emit_p['S'].get(text[-1],0):
            (prob,state) = max([(V[len(text)-1],y) for y in ('E','M')])
        else:
            (prob,state) = max([(V[len(text)-1],y) for y in states])
        
        return (prob,path[state])
    
    #切词函数
    def cut(self,text):
        import os
        if not self.load_para:
            self.try_load_model(os.path.exists(self.model_file))
        prob,pos_list = self.viterbi(text,self.state_list,self.Pi_dic,self.A_dic,self.B_dic)
        begin,next = 0,0
        for i,char in enumerate(text):
            pos = pos_list[i]
            if pos == 'B':
                begin = i
            elif pos == 'E':
                yield text[begin:i+1]
                next = i+1
            elif pos == 'S':
                yield char
                next = i+1
        if next < len(text):
            yield text[next:]

In [ ]:
hmm = HMM()
hmm.train('./data/chapter3/trainCorpus.txt_utf8')
text = '测试一个句子是否能够被正确分词'
#(prob,state) = hmm.viterbi(text,hmm.state_list,hmm.Pi_dic,hmm.A_dic,hmm.B_dic)
res = hmm.cut(text)
#print(state)
print(str(list(res)))

### 其他统计分词算法

<b>条件随机场（CRF）</b>也是一种基于马尔可夫思想的统计模型。   
在HMM中，有一个经典假设，就是每个状态只与它前面的状态有关。这样的假设显然是有偏差的，于是学者们提出了条件随机场算法，使得每个状态不止与他前面的状态有关，还与他后面的状态有关。

## 混合分词

最常用的方法是：先基于词典的方式进行分词，然后再用统计分词方法进行辅助。

## 中文分词工具——jieba

jieba分词结合了基于规则和基于统计这两类方法。

### jieba的三种分词模式

- <b>精确模式</b>：试图将句子最精确地切开，适合文本分析
- <b>全模式</b>：把句子中所有可以成词的词语都扫描出来，速度非常快，但是不能解决歧义
- <b>搜索引擎模式</b>：在精确模式的基础上，对长词再次切分，提高召回率，适用于搜索引擎分词

In [ ]:
import jieba
sent = '中文分词是文本处理不可或缺的一步！'
seg_list = jieba.cut(sent,cut_all=True)
print('全模式：','/'.join(seg_list))
seg_list = jieba.cut(sent,cut_all=False)
print('精确模式：','/'.join(seg_list))
seg_list = jieba.cut(sent)
print('精确模式（默认）：','/'.join(seg_list))
seg_list = jieba.cut_for_search(sent)
print('搜索引擎模式：','/'.join(seg_list))

### 实战之高频词提取

高频词提取其实就是NLP中的TF(Term Frequency)策略，主要有以下干扰项：
- 标点符号
- 停用词

In [ ]:
#定义数据读取函数
def get_content(path):
    with open(path, 'r', encoding='gbk', errors='ignore') as f:
        content = ''
        for l in f:
            l = l.strip()
            content += l
        return content

In [ ]:
#定义高频词统计函数
def get_TF(words,topK=10):
    tf_dic = {}
    for w in words:
        tf_dic[w] = tf_dic.get(w,0) + 1
    return sorted(tf_dic.items(), key=lambda x:x[1],reverse=True)[:topK]

In [ ]:
#定义停用词函数
def stop_words(path):
    with open(path) as f:
        return [l.strip() for l in f]

In [ ]:
def main():
    import glob
    import random
    import jieba

    files = glob.glob('./data/chapter3/news/C000013/*.txt')
    corpus = [get_content(x) for x in files]
    
    sample_inx = random.randint(0,len(corpus))
    split_words = [x for x in jieba.cut(corpus[sample_inx]) if x not in stop_words('./data/chapter3/stop_words.utf8')]
    print('样本之一：'+corpus[sample_inx])
    print('样本分词效果：'+'/'.join(split_words))
    print('样本的topK(10)词：'+str(get_TF(split_words)))

In [ ]:
main()

# 词性标注与命名实体识别

## 词性标注

### jieba分词中的词性标注

In [6]:
import jieba.posseg as psg
sent = '中文分词是文本处理不可或缺的一步！'
seg_list = psg.cut(sent)
#print(str(list(seg_list)))
print(' '.join(['{0}/{1}'.format(w,t) for w,t in seg_list]))

[pair('中文', 'nz'), pair('分词', 'n'), pair('是', 'v'), pair('文本处理', 'n'), pair('不可或缺', 'l'), pair('的', 'uj'), pair('一步', 'm'), pair('！', 'x')]
